In [2]:
import json 
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import time
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame
import numpy as np
import urllib.request
import requests
import itertools
import os
import io
import robin_stocks as robin

In [3]:
interval_check = ['15second', '5minute', '10minute', 'hour', 'day', 'week']
span_check = ['hour', 'day', 'week', 'month', '3month', 'year', '5year']
bounds_check = ['24_7', 'extended', 'regular', 'trading']

In [7]:
f=open("../authentication/account.txt","r")
lines=f.readlines()
username=lines[0].strip()
password=lines[1].strip()
f.close()
login = robin.login(username,password)

In [12]:
crypto=robin.get_crypto_historicals(
    'BTC',
    interval='15second',
    span='hour',
    bounds='24_7',
    info=None,
)

In [19]:
crypto_Mid_prev_stick(crypto,250,-250)

([490.71749999999884,
  595.8550000000032,
  513.3150000000023,
  391.41623300000356,
  315.09249999999884],
 7043.435079999952)

In [16]:
def crypto_Mid_prev_stick(crypto,above,below):
    begins_at=[]
    open_price=[]
    close_price=[]
    high_price=[]
    low_price=[]
    volume=[]
    session=[]
    interpolated=[]
    symbol=[]
    for rs in crypto:
        begins_at.append((rs['begins_at']))
        open_price.append(float(rs['open_price']))
        close_price.append(float(rs['close_price']))
        high_price.append(float(rs['high_price']))
        low_price.append(float(rs['low_price']))
        volume.append(float(rs['volume']))
        session.append((rs['session']))
        interpolated.append(float(rs['interpolated']))
        symbol.append((rs['symbol']))

    df=pd.DataFrame(list(zip(begins_at,open_price,close_price,high_price,low_price,volume,session,interpolated,symbol))
                    ,columns=['begins_at','open_price','close_price','high_price',
                             'low_price','volume','session','interpolated','symbol'])

    df['mid-stick']=(df['high_price']+df['low_price'])/2
    df['mid-stick-error']=df['mid-stick'].diff()

    df['relative-stick']=df['mid-stick'].shift(periods=1)

    df['sell_above']=df['relative-stick']+above
    df['buy_under']=df['relative-stick']+below

    df=df.drop(axis=1,columns=['relative-stick'])

    df['bought'] = np.where(df['buy_under']>= df['low_price'], True, False)
    df['sold'] = np.where(df['sell_above']< df['high_price'], True, False)
    df_chart=df.drop(axis=1,columns=['symbol','interpolated','session','volume','mid-stick','open_price','close_price'])


    w=[]

    global status,position
    status='out'
    position=0

    for i,row in enumerate(df_chart.values):
        if((df_chart.values[i][6]==True) & (df_chart.values[i][7]==True) & (status!='in')):
            w.append(df_chart.values[i][4]-df_chart.values[i][5])
            status='out'
        if((df_chart.values[i][6]==True) & (df_chart.values[i][7]==True) & (status=='in')):
            #we sell
            status='out'
            w.append(df_chart.values[i][4]-position)
            continue
        if((df_chart.values[i][6]==False) & (df_chart.values[i][7]==False)):
            continue
        if((df_chart.values[i][6]==True) & (df_chart.values[i][7]==False) & (status!='in')):
            status='in'
            position=df_chart.values[i][5]
        if((df_chart.values[i][6]==False) & (df_chart.values[i][7]==True) & (status=='in')):
            status='out'
            w.append(df_chart.values[i][4]-position)
            continue
        if((df_chart.values[i][6]==False) & (df_chart.values[i][7]==True) & (status!='in')):
            continue

    return w,np.sum(np.abs(df['mid-stick-error']))

In [417]:
#df.plot(x="begins_at", y=["sell_above", "buy_under",'high_price','low_price'], kind="line",figsize=(16,6))

In [ ]:
up=[]
down=[]
money=[]
for i in range (125,475,25):
    for o in range(-475,-125,25):
        x,y=crypto_Mid_prev_stick(crypto,i,o)
        up.append(i)
        down.append(o)
        money.append(np.sum(x))
        